In [7]:
import json
import numpy as np
from tensorflow.keras.models import load_model
from stgcn_sport2 import STGCN_sport2

keypoints_2 = [
    "Point_0", "Point_7", "Point_11", "Point_12", "Point_23", "Point_24",
    "Point_25", "Point_26", "Point_27", "Point_28", "Point_29", "Point_30",
    "Point_31", "Point_32"
]

def load_json_skeleton_2(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    num_frames = len(data["frames"])
    num_views = 1

    X_data = np.zeros((1, num_frames, num_views, num_joints_2, num_features_2), dtype=np.float32)

    views = ["view4"]

    # ✅ JSON 데이터 -> 배열 변환
    for frame_idx, frame in enumerate(data["frames"]):
        for view_idx, view in enumerate(views):
            pts = frame.get(view, {}).get("pts", {})
            for joint_idx, joint_name in enumerate(keypoints_2):
                if joint_name in pts:
                    X_data[0, frame_idx, view_idx, joint_idx, 0] = pts[joint_name]["x"]
                    X_data[0, frame_idx, view_idx, joint_idx, 1] = pts[joint_name]["y"]

    return X_data

def predict_multiple_json_skeleton(file_paths):
    results = {}


    for file_path in file_paths:
        try:
            # ✅ JSON 데이터 로드
            X_data = load_json_skeleton_2(file_path)

            # ✅ 모델 예측
            prediction = model.predict(X_data)
            
            # ✅ 예측 결과 처리
            predicted_class = np.argmax(prediction, axis=-1)[0]
            confidence = prediction[0][predicted_class]

            # ✅ 결과 저장
            if predicted_class == 0:
                results[file_path] = f"✅ 올바른 자세 ({confidence * 100:.2f}% 확신)"
            else:
                results[file_path] = f"❌ 잘못된 자세 감지 ({confidence * 100:.2f}% 확신)"

        except Exception as e:
            results[file_path] = f"❌ 예측 실패 (오류: {e})"

    return results
    
class LungePostureAnalyzer:
    def __init__(self, model):
        self.model = model
        self.joint_indices = {
            "head": keypoints_2.index("Point_0"),
            "left_shoulder": keypoints_2.index("Point_11"),
            "right_shoulder": keypoints_2.index("Point_12"),
            "left_hip": keypoints_2.index("Point_23"),
            "right_hip": keypoints_2.index("Point_24"),
            "left_knee": keypoints_2.index("Point_25"),
            "right_knee": keypoints_2.index("Point_26"),
            "left_ankle": keypoints_2.index("Point_27"),
            "right_ankle": keypoints_2.index("Point_28"),
        }

    def detect_faulty_posture(self, skeleton_sequence):
        predictions = self.model.predict(skeleton_sequence)
        predicted_label = np.argmax(predictions, axis=-1)[0]
        confidence = predictions[0][predicted_label]

        # ✅ 결과 저장
        if predicted_label == 0:
            result = f"✅ 올바른 자세 ({confidence * 100:.2f}% 확신)"
        else:
            result = f"❌ 잘못된 자세 감지 ({confidence * 100:.2f}% 확신)"
                
        faults = {}
        
        # ✅ 2. 뷰 차원이 1이면 squeeze() 적용
        if skeleton_sequence.shape[2] == 1:
            skeleton_sequence = np.squeeze(skeleton_sequence, axis=2)  # (batch, frames, joints, features)
            
        if predicted_label == 1:  # 잘못된 자세로 분류된 경우
            faults["무릎"] = self.check_knee_angle(skeleton_sequence)
            # faults["몸 방향"] = self.check_alignment(skeleton_sequence)
            faults["상체"] = self.check_torso_tilt(skeleton_sequence)
        
        return {k: v for k, v in faults.items() if v is not None}, result

    def calculate_joint_angle(self, skeleton_sequence, joint_indices):
        a, b, c = [skeleton_sequence[:, :, idx, :] for idx in joint_indices]
        ba = a - b
        bc = c - b
        cosine_angle = np.sum(ba * bc, axis=-1) / (np.linalg.norm(ba, axis=-1) * np.linalg.norm(bc, axis=-1) + 1e-6)
        angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
        return np.degrees(angle)

    def calculate_vector_angle(self, v1, v2):
        dot = np.sum(v1 * v2, axis=-1)
        norm = (np.linalg.norm(v1, axis=-1) * np.linalg.norm(v2, axis=-1)) + 1e-6
        cos_angle = np.clip(dot / norm, -1.0, 1.0)
        angle = np.arccos(cos_angle)
        return np.degrees(angle)

    def check_knee_angle(self, skeleton_sequence, is_left=True):
        hip = self.joint_indices['left_hip'] if is_left else self.joint_indices['right_hip']
        knee = self.joint_indices['left_knee'] if is_left else self.joint_indices['right_knee']
        ankle = self.joint_indices['left_ankle'] if is_left else self.joint_indices['right_ankle']
    
        angles = self.calculate_joint_angle(skeleton_sequence, [hip, knee, ankle])
    
        # 스무딩 적용 (옵션)
        from scipy.signal import savgol_filter
        smoothed_angles = savgol_filter(angles, window_length=5, polyorder=2, axis=-1)
    
        avg_angle = np.mean(smoothed_angles)
    
        print("각 프레임 무릎 각도:", np.round(smoothed_angles, 2))
        print("무릎 평균 각도:", avg_angle)
    
        if not np.sum(smoothed_angles < 160) > 2 or np.sum(smoothed_angles < 100) > 2:
            return f"무릎 각도가 부정확합니다. 90°에 가깝게 유지하세요."
        return None

    # def check_alignment(self, skeleton_sequence, is_left=True):
    #     knee = self.joint_indices['left_knee'] if is_left else self.joint_indices['right_knee']
    #     ankle = self.joint_indices['left_ankle'] if is_left else self.joint_indices['right_ankle']
    #     hip = self.joint_indices['left_hip'] if is_left else self.joint_indices['right_hip']
    #     vec_knee_ankle = skeleton_sequence[:, :, ankle, :] - skeleton_sequence[:, :, knee, :]
    #     vec_hip_knee = skeleton_sequence[:, :, knee, :] - skeleton_sequence[:, :, hip, :]
    #     angle_diff = self.calculate_vector_angle(vec_knee_ankle, vec_hip_knee)
    #     avg_angle = np.mean(angle_diff)
    #     print(angle_diff)
    #     print(avg_angle)
    #     if avg_angle > 30:
    #         return "몸통, 무릎, 발끝의 방향이 일치하지 않습니다. 정렬을 유지하세요."
    #     return None

    def check_torso_tilt(self, skeleton_sequence):
        left_shoulder = self.joint_indices["left_shoulder"]
        left_hip = self.joint_indices["left_hip"]
    
        vec = skeleton_sequence[:, :, left_hip, :] - skeleton_sequence[:, :, left_shoulder, :]  # (batch, frame, 2)
        norm_vec = vec / (np.linalg.norm(vec, axis=-1, keepdims=True) + 1e-6)  # 방향 벡터로 정규화
    
        vertical = np.array([0, 1])  # 완전한 수직선
    
        dot = np.sum(norm_vec * vertical, axis=-1)  # 내적
        angle = np.arccos(np.clip(dot, -1.0, 1.0)) * (180 / np.pi)  # 수직선과의 각도
    
        if np.sum(angle > 10) > 2:
            return "상체가 과도하게 숙이거나 젖혀졌습니다. 몸을 세워주세요."
        return None

    def provide_feedback(self, skeleton_sequence):
        """감지된 자세 오류를 기반으로 실시간 피드백을 제공합니다."""
        faults, result = self.detect_faulty_posture(skeleton_sequence)
        
        print(result)
        
        if not faults:
            return "측정 불가"
        
        feedback = "다음 사항을 수정하세요: "
        for key, message in faults.items():
            feedback += f"\n- {message}"
        
        return feedback
       
# ST-GCN 모델 로드 (미리 저장된 모델 파일 경로)
num_joints_2 = 14 
num_features_2 = 2 
num_classes_2 = 2  
adjacency_matrix_norm_2 = np.load("adjacency_matrix_sport2.npy")

model = STGCN_sport2(num_joints_2, num_features_2, adjacency_matrix_norm_2, num_classes_2)

dummy_input = np.random.rand(1, 10, num_joints_2, num_features_2).astype(np.float32)
model(dummy_input)

model.load_weights("stgcn_model_sport2_1.weights.h5")

In [15]:
analyzer = LungePostureAnalyzer(model)
# file_path = "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_01-7-81.json"
# skeleton_sequence = load_json_skeleton_2(file_path)
# feedback = analyzer.provide_feedback(skeleton_sequence)
# print(feedback)

 
file_paths = [
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_01-7-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_01-8-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_02-7-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_02-8-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_04-8-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_04-9-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_05-8-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_05-9-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_14-8-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_14-9-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_16-8-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_16-9-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_16-7-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_14-7-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_05-7-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_04-7-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_02-6-81.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_01-1-91.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_01-1-92.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_01-1-93.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_01-1-94.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_02-1-82.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_02-1-83.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_02-1-84.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_02-1-85.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_04-1-86.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_04-1-87.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_04-1-88.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_04-1-89.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_05-1-90.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_05-1-91.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_05-1-92.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_05-1-93.json",
    # "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/body_05-1-94.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/sport2/123123.json",
    
]

for file_path in file_paths:
    print(file_path)
    skeleton_sequence = load_json_skeleton_2(file_path)
    feedback = analyzer.provide_feedback(skeleton_sequence)
    print(feedback)
    print()
    print()

D:/Studying/gradu/013.피트니스자세/1.Training/sport2/123123.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
✅ 올바른 자세 (100.00% 확신)
측정 불가


